# **Default Setting**

In [6]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Load Dataset**

In [2]:
import numpy as np

# check files
kor_dataset = np.load("/content/drive/My Drive/Speaker Recognition/dataset/kor_dataset.npz")
eng_dataset = np.load("/content/drive/My Drive/Speaker Recognition/dataset/eng_dataset.npz")

print("kor_dataset[\"data_{}\"].shape:\t\t{}".format(3 ** 8, kor_dataset["data_{}".format(3 ** 8)].shape))
print("kor_dataset[\"data_{}\"].shape:\t{}".format(2 * (3 ** 8), kor_dataset["data_{}".format(2 * (3 ** 8))].shape))
print("kor_dataset[\"data_{}\"].shape:\t{}".format(3 ** 9, kor_dataset["data_{}".format(3 ** 9)].shape))
print("kor_dataset[\"labels\"].shape:\t\t{}".format(kor_dataset["labels"].shape))

print("eng_dataset[\"data_{}\"].shape:\t\t{}".format(3 ** 8, eng_dataset["data_{}".format(3 ** 8)].shape))
print("eng_dataset[\"data_{}\"].shape:\t{}".format(2 * (3 ** 8), eng_dataset["data_{}".format(2 * (3 ** 8))].shape))
print("eng_dataset[\"data_{}\"].shape:\t{}".format(3 ** 9, eng_dataset["data_{}".format(3 ** 9)].shape))
print("eng_dataset[\"labels\"].shape:\t\t{}".format(eng_dataset["labels"].shape))

kor_dataset["data_6561"].shape:		(50, 100, 6561)
kor_dataset["data_13122"].shape:	(50, 100, 13122)
kor_dataset["data_19683"].shape:	(50, 100, 19683)
kor_dataset["labels"].shape:		(50, 100)
eng_dataset["data_6561"].shape:		(50, 100, 6561)
eng_dataset["data_13122"].shape:	(50, 100, 13122)
eng_dataset["data_19683"].shape:	(50, 100, 19683)
eng_dataset["labels"].shape:		(50, 100)


# **Create Model**

In [0]:
def create_NN1(model_name, data_length, ways = 5):
    
    model_input = tf.keras.layers.Input((data_length, 1))

    # block 1
    x = tf.keras.layers.Conv1D(name = "block1_conv", filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(model_input)
    x = tf.keras.layers.BatchNormalization(name = "block1_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block1_pool", pool_size = 3)(x)

    # block 2
    x = tf.keras.layers.Conv1D(name = "block2_conv", filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block2_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block2_pool", pool_size = 3)(x)

    # block 3
    x = tf.keras.layers.Conv1D(name = "block3_conv", filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block3_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block3_pool", pool_size = 3)(x)

    x = tf.keras.layers.GlobalAveragePooling1D(name = "avg_pool")(x)
    # x = tf.keras.layers.Dense(name = "dense", units = 128, activation = None)(x)

    # out = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis = 1), name = "l2_norm_embedding")(x) # L2 normalize embeddings
    out = tf.keras.layers.Dense(units = ways, name = "dense", activation = "softmax")(x)

    model = tf.keras.models.Model(
        inputs = model_input,
        outputs = out,
        name = model_name)
    
    return model

In [0]:
def create_NN2(model_name, data_length, ways = 5):
    
    model_input = tf.keras.layers.Input((data_length, 1))

    # block 1
    x = tf.keras.layers.Conv1D(name = "block1_conv", filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(model_input)
    x = tf.keras.layers.BatchNormalization(name = "block1_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block1_pool", pool_size = 3)(x)

    # block 2
    x = tf.keras.layers.Conv1D(name = "block2_conv", filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block2_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block2_pool", pool_size = 3)(x)

    # block 3
    x = tf.keras.layers.Conv1D(name = "block3_conv", filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.BatchNormalization(name = "block3_bn")(x)
    x = tf.keras.layers.MaxPooling1D(name = "block3_pool", pool_size = 3)(x)

    x = tf.keras.layers.GlobalAveragePooling1D(name = "avg_pool")(x)
    x = tf.keras.layers.Dense(name = "dense", units = 128, activation = None)(x)

    out = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis = 1), name = "l2_norm_embedding")(x) # L2 normalize embeddings
    # out = tf.keras.layers.Dense(units = ways, name = "dense", activation = "softmax")(x)

    model = tf.keras.models.Model(
        inputs = model_input,
        outputs = out,
        name = model_name)
    
    return model

In [7]:
tmp_model_1 = create_NN1("tmp_model_1", 3 ** 9)
tmp_model_1.summary()

Model: "tmp_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 19683, 1)]        0         
_________________________________________________________________
block1_conv (Conv1D)         (None, 19683, 32)         128       
_________________________________________________________________
block1_bn (BatchNormalizatio (None, 19683, 32)         128       
_________________________________________________________________
block1_pool (MaxPooling1D)   (None, 6561, 32)          0         
_________________________________________________________________
block2_conv (Conv1D)         (None, 6561, 64)          6208      
_________________________________________________________________
block2_bn (BatchNormalizatio (None, 6561, 64)          256       
_________________________________________________________________
block2_pool (MaxPooling1D)   (None, 2187, 64)          

In [8]:
tmp_model_2 = create_NN2("tmp_model_2", 3 ** 9)
tmp_model_2.summary()

Model: "tmp_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 19683, 1)]        0         
_________________________________________________________________
block1_conv (Conv1D)         (None, 19683, 32)         128       
_________________________________________________________________
block1_bn (BatchNormalizatio (None, 19683, 32)         128       
_________________________________________________________________
block1_pool (MaxPooling1D)   (None, 6561, 32)          0         
_________________________________________________________________
block2_conv (Conv1D)         (None, 6561, 64)          6208      
_________________________________________________________________
block2_bn (BatchNormalizatio (None, 6561, 64)          256       
_________________________________________________________________
block2_pool (MaxPooling1D)   (None, 2187, 64)          

# **Compile and Train model**

In [0]:
import tensorflow_addons as tfa

def compile_model(model):
    if "NN1" in model.name:
        loss = "categorical_crossentropy"
    elif "NN2" in model.name:
        loss = tfa.losses.TripletSemiHardLoss()
    else:
        print("wrong model name..")
        return

    model.compile(
        optimizer = tf.keras.optimizers.Adam(0.01),
        loss = loss)

def scheduler(epoch, lr):
    # lr = 0.5 * lr for every 30 epochs
    if not epoch:
        return lr
    elif not (epoch % 30):
        return 0.5 * lr
    else:
        return lr

def train_model(model, train_dataset, epochs):
    history = model.fit(
        train_dataset, verbose = 0, epochs = epochs,
        callbacks = [tf.keras.callbacks.LearningRateScheduler(scheduler)])
    return history

In [0]:
def split_dataset(kor_data, kor_labels, eng_data, eng_laels, data_length, kor_shot, 
                  kor_shots_labels, eng_shots_labels, way = 5, shot = 5,  max_shot = 20):

    train_data = np.concatenate((
        np.reshape(kor_data[:ways, kor_shots_labels, :], (-1, data_length)),
        np.reshape(eng_data[:ways, eng_shots_labels, :], (-1, data_length))),
        axis = 0)
    train_labels = np.concatenate((
        np.reshape(kor_labels[:ways, kor_shots_labels], (-1)),
        np.reshape(eng_labels[:ways, eng_shots_labels], (-1))),
        axis = 0)
    
    test_data = np.concatenate((
        np.reshape(kor_data[:ways, max_shot:, :], (-1, data_length)),
        np.reshape(eng_data[:ways, max_shot:, :], (-1, data_length))),
        axis = 0)
    test_labels = np.concatenate((
        np.reshape(kor_labels[:ways, max_shot:], (-1)),
        np.reshape(eng_labels[:ways, max_shot:], (-1))),
        axis = 0)
    
    # print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)
    return train_data, train_labels, test_data, test_labels

# **Generate 360 models (3 x 6 x 10 x 2) in 5-shot**
*   data length : [3^8, 2x3^8, 3^9]
*   kor_shot / eng_shot : [K0E5, K1E4, K2E3, K3E2, K4E1, K5E0]
*   repeats : 10 times
*   model type : [Neural Network 1, Neural Network 2]
    *   Neural Network 1 : use categorical_crossentropy loss func
    *   Neural Network 2 : use triplet semi-hard loss function

In [11]:
%%time
import os
import tensorflow as tf

ways = 5
shots = 5
max_shot = 20
repeats = 10

epochs = 100
batch_size = 128

random_states = np.zeros((3, 6, 10, 2), dtype = np.object)

save_path = "/content/drive/My Drive/Speaker Recognition/models"
if not os.path.exists(save_path):
    os.mkdir(save_path)

for i, data_length in enumerate([3**8, 2 * (3**8), 3**9]):
    kor_data = kor_dataset["data_{}".format(data_length)]
    kor_labels = kor_dataset["labels"]

    eng_data = eng_dataset["data_{}".format(data_length)]
    eng_labels = eng_dataset["labels"]
    
    for ks in range(5 + 1):
        for repeat in range(repeats):
            # choose and save random state
            kor_shots_labels = np.random.choice(max_shot, size = ks, replace = False)
            eng_shots_labels = np.random.choice(max_shot, size = shots - ks, replace = False)

            random_states[i, ks, repeat, 0] = kor_shots_labels
            random_states[i, ks, repeat, 1] = eng_shots_labels

            print(random_states[i, ks, repeat, 0], random_states[i, ks, repeat, 1])

            # split dataset for every repeats
            train_data, train_labels, _, _ = split_dataset(
                kor_data, kor_labels, eng_data, eng_labels, data_length, ks, kor_shots_labels, eng_shots_labels)

            train_dataset = tf.data.Dataset.from_tensor_slices(
                (train_data[:, :, np.newaxis], train_labels))
            train_dataset = train_dataset.shuffle(buffer_size = data_length * 1000).batch(batch_size)

            one_hot_train_dataset = tf.data.Dataset.from_tensor_slices(
                (train_data[:, :, np.newaxis], tf.keras.utils.to_categorical(train_labels)))
            one_hot_train_dataset = one_hot_train_dataset.shuffle(buffer_size = data_length * 1000).batch(batch_size)

            # NN1
            model_NN1_name = "{}_{}_{}_K{}E{}_{}_{}".format(ways, shots, data_length, ks, shots - ks, "NN1", repeat)
            print(model_NN1_name)
            model_NN1 = create_NN1(model_NN1_name, data_length)

            compile_model(model_NN1)
            train_model(model_NN1, one_hot_train_dataset, epochs)
            model_NN1.save(os.path.join(save_path, model_NN1_name + ".h5"))

            # NN2
            model_NN2_name = "{}_{}_{}_K{}E{}_{}_{}".format(ways, shots, data_length, ks, shots - ks, "NN2", repeat)
            print(model_NN2_name)
            model_NN2 = create_NN2(model_NN2_name, data_length)

            compile_model(model_NN2)
            train_model(model_NN2, train_dataset, epochs)
            model_NN2.save(os.path.join(save_path, model_NN2_name + ".h5"))
    

[] [ 1  7 15  2  4]
5_5_6561_K0E5_NN1_0
5_5_6561_K0E5_NN2_0
[] [13 18 17  7  1]
5_5_6561_K0E5_NN1_1
5_5_6561_K0E5_NN2_1
[] [ 8  1 14  4  9]
5_5_6561_K0E5_NN1_2
5_5_6561_K0E5_NN2_2
[] [ 1  0 16  3  6]
5_5_6561_K0E5_NN1_3
5_5_6561_K0E5_NN2_3
[] [18 12  5 11 15]
5_5_6561_K0E5_NN1_4
5_5_6561_K0E5_NN2_4
[] [ 8  0 12 18 14]
5_5_6561_K0E5_NN1_5
5_5_6561_K0E5_NN2_5
[] [16  2  0  1 12]
5_5_6561_K0E5_NN1_6
5_5_6561_K0E5_NN2_6
[] [16  0 11  7 10]
5_5_6561_K0E5_NN1_7
5_5_6561_K0E5_NN2_7
[] [ 9 10 14  4 16]
5_5_6561_K0E5_NN1_8
5_5_6561_K0E5_NN2_8
[] [15  2 14  3  4]
5_5_6561_K0E5_NN1_9
5_5_6561_K0E5_NN2_9
[1] [ 5  7 11  9]
5_5_6561_K1E4_NN1_0
5_5_6561_K1E4_NN2_0
[1] [17  9  8  2]
5_5_6561_K1E4_NN1_1
5_5_6561_K1E4_NN2_1
[18] [ 4 17  3  0]
5_5_6561_K1E4_NN1_2
5_5_6561_K1E4_NN2_2
[19] [18 15  1  7]
5_5_6561_K1E4_NN1_3
5_5_6561_K1E4_NN2_3
[4] [18 16 17  7]
5_5_6561_K1E4_NN1_4
5_5_6561_K1E4_NN2_4
[19] [17 12 10  1]
5_5_6561_K1E4_NN1_5
5_5_6561_K1E4_NN2_5
[4] [12 11  2 19]
5_5_6561_K1E4_NN1_6
5_5_6561_K1

TypeError: ignored

In [0]:
# save random states
np.save(
    os.path.join(save_path, "random_states"),
    random_states)